# 导入库

In [1]:
import tensorflow as tf
import keras 
import cv2
import os
import scipy
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
import os
import h5py
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

Using TensorFlow backend.


# 图片命名

In [ ]:
#coding:utf8
import os;

def rename():
         i=0
         path="picture//牛鲨";
         filelist=os.listdir(path)#该文件夹下所有的文件（包括文件夹）
         for files in filelist:#遍历所有文件
             i=i+1
             Olddir=os.path.join(path,files);#原来的文件路径                
             if os.path.isdir(Olddir):#如果是文件夹则跳过
                     continue;
             filename=os.path.splitext(files)[0];#文件名
             filetype=os.path.splitext(files)[1];#文件扩展名
             fn = "bull_"
             Newdir=os.path.join(path,fn+str(i)+filetype);#新的文件路径
             os.rename(Olddir,Newdir)#重命名
rename()

# 数据处理

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.15,
    height_shift_range=0.15,
    zoom_range=0.15,
    shear_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')
dirs = os.listdir("allshark")
print(len(dirs))
for filename in dirs:
    img = load_img("allshark//{}".format(filename))
    x = img_to_array(img)
    # print(x.shape)
    x = x.reshape((1,) + x.shape) #datagen.flow要求rank为4
    # print(x.shape)
    datagen.fit(x)
    prefix = filename.split('.')[0]
    print(prefix)
    counter = 0
    for batch in datagen.flow(x, batch_size=4 , save_to_dir='generater_pic', save_prefix=prefix, save_format='jpg'):
        counter += 1
        if counter > 100:
            break  # 否则生成器会退出循环

In [2]:
def resize_img():
    dirs = os.listdir("generater_pic")
    for filename in dirs:
        im = tf.gfile.FastGFile("generater_pic//{}".format(filename), 'rb').read()
        # print("正在处理第%d张照片"%counter)
        with tf.Session() as sess:
            img_data = tf.image.decode_jpeg(im)
            image_float = tf.image.convert_image_dtype(img_data, tf.float32)
            resized = tf.image.resize_images(image_float, [64, 64], method=3)
            resized_im = resized.eval()
            # new_mat = np.asarray(resized_im).reshape(1, 64, 64, 3)
            scipy.misc.imsave("resized_img//{}".format(filename),resized_im)

In [ ]:
resize_img() 

/private/sxu/.conda/envs/Tensorflow-gpu/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  if sys.path[0] == '':


In [2]:
#图片转h5文件
    #test
    # data = h5py.File("dataset//data.h5","r")
    # X_data = data['X']
    # print(X_data.shape)
    # Y_data = data['Y']
    # print(Y_data[123])
    # image = Image.fromarray(X_data[123]) #矩阵转图片并显示
    # image.show()

In [ ]:
dirs = os.listdir("resized_img")
Y = [] #label
X = [] #data
i = 0
print(len(dirs))
for filename in dirs:
    print(i/len(dirs))
    i = i+1
    label = str(filename.split('_')[0]).encode()
    #print(filename)
    #print(label)
    Y.append(label)
    im = Image.open("resized_img//{}".format(filename)).convert('RGB')
    mat = np.asarray(im) #image 转矩阵
    X.append(mat)


In [ ]:
file = h5py.File("dataset//data.h5","w")
file.create_dataset('X', data=np.array(X))
file.create_dataset('Y', data=np.array(Y))
file.close()

In [ ]:
data = h5py.File("dataset//data.h5","r")
X_data = data['X']
print(X_data.shape)
Y_data = data['Y']
print(Y_data[1])
image = Image.fromarray(X_data[1]) #矩阵转图片并显示
image.show()

In [ ]:
#def load_dataset():
#划分训练集、测试集
data = h5py.File("dataset//data.h5","r")
X_data = np.array(data['X']) #data['X']是h5py._hl.dataset.Dataset类型，转化为array
Y_data = np.array(data['Y'])
# print(type(X_data))
X_train, X_test, y_train, y_test = train_test_split(X_data, Y_data, train_size=0.9, test_size=0.1, random_state=22)

In [60]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(8667, 64, 64, 3)
(8667, 2)
(963, 64, 64, 3)
(963, 2)


In [2]:
def image_to_h5():
    dirs = os.listdir("resized_img")
    Y = [] #label
    X = [] #data
    i = 0
    print(len(dirs))
    for filename in dirs:
        print(i/len(dirs))
        i = i+1
        label = str(filename.split('_')[0]).encode()
        Y.append(label)
        im = Image.open("resized_img//{}".format(filename)).convert('RGB')
        mat = np.asarray(im) #image 转矩阵
        X.append(mat)

    file = h5py.File("dataset//data.h5","w")
    file.create_dataset('X', data=np.array(X))
    file.create_dataset('Y', data=np.array(Y))
    file.close()

In [ ]:
image_to_h5()

In [4]:
def load_dataset():
    data = h5py.File("dataset//data.h5","r")
    X_data = np.array(data['X']) #data['X']是h5py._hl.dataset.Dataset类型，转化为array
    Y_data = np.array(data['Y'])
    # print(type(X_data))
    X_train, X_test, y_train, y_test = train_test_split(X_data, Y_data, train_size=0.9, test_size=0.1, random_state=22)
    # print(X_train.shape)
    # print(y_train[456])
    # image = Image.fromarray(X_train[456])
    # image.show()
    # y_train = y_train.reshape(1,y_train.shape[0])
    # y_test = y_test.reshape(1,y_test.shape[0])
    # print(X_train[0])
    X_train = X_train / 255.  # 归一化
    X_test = X_test / 255.
    # print(X_train[0])
    # one-hot
    encoder = LabelEncoder()
    train_Y = encoder.fit_transform(y_train)
    test_Y = encoder.fit_transform(y_test)
    y_train = np_utils.to_categorical(train_Y)
    y_test = np_utils.to_categorical(test_Y)
    print(X_train.shape)
    print(y_train.shape)
    print(X_test.shape)
    print(y_test.shape)

    return X_train, X_test, y_train, y_test

In [5]:
X_train, X_test, y_train, y_test = load_dataset()

(8667, 64, 64, 3)
(8667, 2)
(963, 64, 64, 3)
(963, 2)


In [8]:
print(type(y_train))
y_train[0:15]

<class 'numpy.ndarray'>


array([[ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.]], dtype=float32)

# Public function

In [9]:
def weight_variable(shape):
    tf.set_random_seed(1)
    return tf.Variable(tf.truncated_normal(shape, stddev=0.1))

In [10]:
def bias_variable(shape):
    return tf.Variable(tf.constant(0.0, shape=shape))

In [11]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

In [12]:
def max_pool_2x2(z):
    return tf.nn.max_pool(z, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [13]:
def random_mini_batches(X, Y, mini_batch_size=16, seed=0):
    """
    Creates a list of random minibatches from (X, Y)

    Arguments:
    X -- input data, of shape (input size, number of examples)
    Y -- true "label" vector (containing 0 if cat, 1 if non-cat), of shape (1, number of examples)
    mini_batch_size - size of the mini-batches, integer
    seed -- this is only for the purpose of grading, so that you're "random minibatches are the same as ours.

    Returns:
    mini_batches -- list of synchronous (mini_batch_X, mini_batch_Y)
    """

    m = X.shape[0]  # number of training examples
    mini_batches = []
    np.random.seed(seed)

    # Step 1: Shuffle (X, Y)
    permutation = list(np.random.permutation(m))
    shuffled_X = X[permutation]
    shuffled_Y = Y[permutation,:].reshape((m, Y.shape[1]))

    # Step 2: Partition (shuffled_X, shuffled_Y). Minus the end case.
    num_complete_minibatches = math.floor(m / mini_batch_size)  # number of mini batches of size mini_batch_size in your partitionning
    for k in range(0, num_complete_minibatches):
        mini_batch_X = shuffled_X[k * mini_batch_size: k * mini_batch_size + mini_batch_size]
        mini_batch_Y = shuffled_Y[k * mini_batch_size: k * mini_batch_size + mini_batch_size]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)

    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:
        mini_batch_X = shuffled_X[num_complete_minibatches * mini_batch_size: m]
        mini_batch_Y = shuffled_Y[num_complete_minibatches * mini_batch_size: m]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)

    return mini_batches


# Input Layer

In [14]:
with tf.name_scope('Input_Layer'):
    X = tf.placeholder(tf.float32, [None, 64, 64, 3], name="input_x")
    y = tf.placeholder(tf.float32, [None, 2], name="input_y")
    kp = tf.placeholder_with_default(1.0, shape=(), name="keep_prob")
    lam = tf.placeholder(tf.float32, name="lamda")

# Convolutional Layer 1

In [15]:
with tf.name_scope('C1_Conv'):
    W_conv1 = weight_variable([5,5,3,32])
    b_conv1 = bias_variable([32])
    z1 = tf.nn.relu(conv2d(X, W_conv1) + b_conv1)

In [16]:
with tf.name_scope('C1_Pool'):
    maxpool1 = max_pool_2x2(z1) #max_pool1完后maxpool1维度为[?,32,32,32]

# Convolutional Layer 2

In [17]:
with tf.name_scope('C2_Conv'):
    W_conv2 = weight_variable([5,5,32,64])
    b_conv2 = bias_variable([64])
    z2 = tf.nn.relu(conv2d(maxpool1, W_conv2) + b_conv2)

In [18]:
with tf.name_scope('C2_Pool'):
    maxpool2 = max_pool_2x2(z2) #max_pool2,shape [?,16,16,64]

# Convolutional Layer 3

In [19]:
with tf.name_scope('C3_Conv'):
    W_conv3 = weight_variable([5, 5, 64, 128])
    b_conv3 = bias_variable([128])
    z3 = tf.nn.relu(conv2d(maxpool2, W_conv3) + b_conv3)

In [20]:
with tf.name_scope('C3_Pool'):
    maxpool3 = max_pool_2x2(z3)  # max_pool3,shape [?,8,8,128]

# Fully Connected Layer

In [21]:
with tf.name_scope('D_Flat'):
    maxpool3_flat = tf.reshape(maxpool3, [-1, 8*8*128])

In [22]:
with tf.name_scope('D_Hidden_Layer'):    
    W_fc1 = weight_variable([8*8*128, 200])
    b_fc1 = bias_variable([200])
    z_fc1 = tf.nn.relu(tf.matmul(maxpool3_flat, W_fc1) + b_fc1)
    z_fc1_drop = tf.nn.dropout(z_fc1, keep_prob=kp)            #Drop_out kp=0.8

# Output  Layer

In [29]:
with tf.name_scope('Output_Layer'):
    W_fc2 = weight_variable([200, 2])
    b_fc2 = bias_variable([2])
    z_fc2 = tf.add(tf.matmul(z_fc1_drop, W_fc2),b_fc2, name="outlayer")
    prob = tf.nn.softmax(z_fc2, name="probability")

# Optimazer

In [30]:
with tf.name_scope("optimizer"):
    regularizer = tf.contrib.layers.l2_regularizer(lam)       # L2正则 lam=0.5
    regularization = regularizer(W_fc1) + regularizer(W_fc2)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=z_fc2)) + regularization
    train = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

# Evaluate Model

In [25]:
with tf.name_scope("evaluate_model"):
    pred = tf.argmax(prob, 1, output_type="int32", name="predict")  # 输出结点名称predict方便后面保存为pb文件
    correct_prediction = tf.equal(pred, tf.argmax(y, 1, output_type='int32'))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [26]:
epoch_list=[];loss_list=[];
from time import time
startTime=time()

In [31]:
import math
def cnn_model(X_train, y_train, X_test, y_test, keep_prob, lamda=0.5, num_epochs = 450, minibatch_size = 20):
    init = tf.global_variables_initializer()
    tf.set_random_seed(1)  # to keep consistent results
    seed = 0
    with tf.Session() as sess:
        sess.run(init)
        for epoch in range(num_epochs):
            seed = seed + 1
            epoch_cost = 0.
            num_minibatches = int(X_train.shape[0] / minibatch_size)
            minibatches = random_mini_batches(X_train, y_train, minibatch_size, seed)
            start_epo = time()
            for minibatch in minibatches:
                (minibatch_X, minibatch_Y) = minibatch
                _, minibatch_cost = sess.run([train, cost], feed_dict={X: minibatch_X, y: minibatch_Y, kp: keep_prob, lam: lamda})

                epoch_cost += minibatch_cost / num_minibatches

            epoch_list.append(epoch)
            loss_list.append(epoch_cost)
            end_epo = time() - start_epo
            if epoch % 10 == 0:
                print("Cost after epoch %i: %f spend %d s" % (epoch, epoch_cost, end_epo))

        train_acc = accuracy.eval(feed_dict={X: X_train, y: y_train, kp: 0.8, lam: lamda})
        test_acc = accuracy.eval(feed_dict={X: X_test, y: y_test, lam: lamda})
        print("train accuracy", train_acc)
        print("test accuracy", test_acc)
        
        duration =time()-startTime
        print("Train Finished takes:",duration) 

tf.reset_default_graph()

In [32]:
cnn_model(X_train, y_train, X_test, y_test,keep_prob=0.8)

Cost after epoch 0: 372.312835 
Cost after epoch 10: 0.148621 
Cost after epoch 20: 0.083549 
Cost after epoch 30: 0.122189 
Cost after epoch 40: 0.061700 
Cost after epoch 50: 0.162271 
Cost after epoch 60: 0.112326 
Cost after epoch 70: 0.074159 
Cost after epoch 80: 0.049136 
Cost after epoch 90: 0.086770 
Cost after epoch 100: 0.060377 
Cost after epoch 110: 0.021280 
Cost after epoch 120: 0.131438 
Cost after epoch 130: 0.165240 
Cost after epoch 140: 0.018229 
Cost after epoch 150: 0.112668 
Cost after epoch 160: 0.214789 
Cost after epoch 170: 0.098081 
Cost after epoch 180: 0.078775 
Cost after epoch 190: 0.018031 
Cost after epoch 200: 0.017662 
Cost after epoch 210: 0.154246 
Cost after epoch 220: 0.017465 
Cost after epoch 230: 0.071084 
Cost after epoch 240: 0.041998 
Cost after epoch 250: 0.245037 
Cost after epoch 260: 0.321798 
Cost after epoch 270: 0.012690 
Cost after epoch 280: 0.023441 
Cost after epoch 290: 0.014912 
Cost after epoch 300: 0.032601 
Cost after epoch 

ResourceExhaustedError: OOM when allocating tensor with shape[8667,32,64,64] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: C1_Conv/Conv2D = Conv2D[T=DT_FLOAT, data_format="NCHW", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](C1_Conv/Conv2D-0-TransposeNHWCToNCHW-LayoutOptimizer, C1_Conv/Variable/read)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: evaluate_model/Mean/_29 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_92_evaluate_model/Mean", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of allocated tensors when 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
fig = plt.gcf()
fig.set_size_inches(4,2)
plt.plot(epoch_list, loss_list, label = 'loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['loss'], loc='upper left')

#save model
saver = tf.train.Saver({'W_conv1':W_conv1, 'b_conv1':b_conv1, 'W_conv2':W_conv2, 'b_conv2':b_conv2,
                        'W_fc1':W_fc1, 'b_fc1':b_fc1, 'W_fc2':W_fc2, 'b_fc2':b_fc2})
saver.save(sess, "model_500_200_c3//cnn_model.ckpt")

#将训练好的模型保存为.pb文件，方便在Android studio中使用
output_graph_def = graph_util.convert_variables_to_constants(sess, sess.graph_def, output_node_names=['predict'])

with tf.gfile.FastGFile('model_500_200_c3//digital_gesture.pb', mode='wb') as f:  # ’wb’中w代表写文件，b代表将数据以二进制方式写入文件。
    f.write(output_graph_def.SerializeToString())